In [1]:
from mlp.utils import test_conv_linear_fprop
from mlp.conv import ConvLinear

#from mlp.conv import ConvLinear

layer = ConvLinear(3,2,kernel_shape=(2,2))
test_conv_linear_fprop(layer)

True

In [2]:
from mlp.utils import test_conv_linear_bprop
from mlp.conv import ConvLinear

layer = ConvLinear(3,2, kernel_shape=(2,2))
test_conv_linear_bprop(layer)

True

In [3]:
from mlp.utils import test_conv_linear_pgrads
from mlp.conv import ConvLinear

layer = ConvLinear(3,2, kernel_shape=(2,2))
test_conv_linear_pgrads(layer)

True

In [1]:
from mlp.utils import test_maxpool_fprop,test_maxpool_generic_fprop
from mlp.conv import ConvMaxPool2D

layer = ConvMaxPool2D(2, (4,4), pool_shape=(2,2), pool_stride=(2,2))
test_maxpool_fprop(layer)
layer = ConvMaxPool2D(2, (4,4), pool_shape=(2,2), pool_stride=(1,1))
test_maxpool_generic_fprop(layer)



[[[[  1.   0.   2.]
   [  3.   4.   4.]
   [  3.   4.   4.]]

  [[  7.   0.   8.]
   [  3.   5.   5.]
   [  6.   6.   5.]]]


 [[[ 10.   0.   2.]
   [  3.   4.   4.]
   [  3.   4.   4.]]

  [[  8.   0.  11.]
   [  9.   9.   5.]
   [ 10.   9.   5.]]]]
[[[[  1.   0.   2.]
   [  3.   4.   4.]
   [  3.   4.   4.]]

  [[  7.   0.   8.]
   [  3.   5.   5.]
   [  6.   6.   5.]]]


 [[[ 10.   0.   2.]
   [  3.   4.   4.]
   [  3.   4.   4.]]

  [[  8.   0.  11.]
   [  9.   9.   5.]
   [ 10.   9.   5.]]]]


True

In [1]:
from mlp.utils import test_maxpool_bprop, test_maxpool_generic_bprop
from mlp.conv import ConvMaxPool2D

layer = ConvMaxPool2D(2,(24,24), pool_stride=(2,2))
test_maxpool_bprop(layer)

layer = ConvMaxPool2D(2,(24,24), pool_stride=(1,1))
test_maxpool_generic_bprop(layer)


True

In [6]:
import numpy

x = numpy.ones((1,1,2,2))
print x

print x+1

'''
Verify Gradients - WHY THE FUCK WON'T IT WORK

Cython 
Experiments
'''

[[[[ 1.  1.]
   [ 1.  1.]]]]
[[[[ 2.  2.]
   [ 2.  2.]]]]


'\nVerify Gradients\nMake checks - Done maxpool Checks\nMake maxpool generic, just change bprop?\n\nCython \nExperiments\n'

In [7]:
#Do back prop check for maxout

In [8]:
#Do sigmoid checks

In [9]:
#Do Relu checks

In [10]:
from mlp.layers import max_and_argmax
import numpy

y = numpy.array(
      [[[[  496.,   500.,   436.],
         [  376.,   500.,   316.],
         [  256.,   226.,   500.]],
        [[ 1385.,  1403.,  1421.],
         [ 1457.,  1475.,  1493.],
         [ 1529.,  1547.,  1565.]]],
       [[[ -944.,  -974., -1004.],
         [-1064., -1094., -1124.],
         [-1184., -1214., -1244.]],
        [[ 2249.,  2267.,  2285.],
         [ 2321.,  2339.,  2357.],
         [ 2393.,  2411.,  2429.]]]])


mx, mxi = max_and_argmax(y[0,0,:,:],keepdims_argmax=True)
G = numpy.zeros(y.shape)
G[0,0,mxi[0],mxi[1]] = 1

In [11]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = []  # remove any existing handlers
file_handler = logging.FileHandler('scl.log')
file_handler.setFormatter(logging.Formatter(
    '%(name)s:%(levelname)s %(message)s'))
logger.addHandler(file_handler)

In [5]:
%load_ext cython



In [9]:
%%cython

import numpy
cimport numpy

from mlp.layers import max_and_argmax

from types import *


DTYPE = numpy.float32
ctypedef numpy.float32_t DTYPE_t


DTYPE2 = numpy.int
ctypedef numpy.int_t DTYPE2_t

def my1_conv2d(numpy.ndarray[DTYPE_t, ndim=4] image,numpy.ndarray[DTYPE_t, ndim=4] kernels, strides=(1, 1), pool=False):
    """
    Implements a 2d valid convolution of kernels with the image
    Note: filter means the same as kernel and convolution (correlation) of those with the input space
    produces feature maps (sometimes refered to also as receptive fields). Also note, that
    feature maps are synonyms here to channels, and as such num_inp_channels == num_inp_feat_maps
    :param image: 4D tensor of sizes (batch_size, num_input_channels, img_shape_x, img_shape_y)
    :param filters: 4D tensor of filters of size (num_inp_feat_maps, num_out_feat_maps, kernel_shape_x, kernel_shape_y)
    :param strides: a tuple (stride_x, stride_y), specifying the shift of the kernels in x and y dimensions
    :return: 4D tensor of size (batch_size, num_out_feature_maps, feature_map_shape_x, feature_map_shape_y)
    """
   
    #Get batch_size out
    cdef int batch_size = image.shape[0]
    
    #Get the kernels size
    cdef int out = kernels.shape[1]
    
    '''
    
    Image dims - Used equation from stanford lectures: http://cs231n.github.io/convolutional-networks/
    if((InputSize-FilterSize+2Padding)/Stride+1) is valid int then carry on else throw and error, first calculate
    '''
    
    #Padding not implemented
    cdef int padding = 0
    
    cdef int kxdim = kernels.shape[2]
    cdef int kydim = kernels.shape[3]
    #Get feature map size out
    cdef int num_out_feat_maps = kernels.shape[1]
    #Get input feature size
    cdef int num_inp_feat_maps = kernels.shape[0]
        
    
    assert kxdim <= image.shape[2], 'Kernel can be max the size of the image'
    assert kydim <= image.shape[3], 'Kernel can be max the size of the image'
    
    #Can calculate here as this is all we are going to go to anyway - xdims is how many times kernel can move...
    #These only tell us if we can use strides
    cdef int strides_pos_x = ((image.shape[2] - kxdim + (2*padding))/(strides[0])) +1
    cdef int strides_pos_y = ((image.shape[3] - kydim + (2*padding))/(strides[1])) +1
    
    #Do assertions to ensure passed in the correct type, strides pos also correspond to the output size.
    assert strides_pos_x.dtype == DTYPE2,"Can't make feature map with x-stride: %r" % strides[0]
    assert strides_pos_y.dtype == DTYPE2,"Can't make feature map with y-stride: %r" % strides[1]
    
    #Create G matrix
    cdef numpy.ndarray G = numpy.zeros((image.shape[0],image.shape[1],image.shape[2],image.shape[3]))
    
    #Actual number of dimensions to traverse, +1 to count for range function
    cdef int xdims = (image.shape[2] - kxdim) + 1
    cdef int ydims = (image.shape[3] - kydim) + 1
    
    #Create empty 4D tensor
    cdef numpy.ndarray output =  numpy.zeros((batch_size,out,strides_pos_x,strides_pos_y), dtype=DTYPE)
    cdef numpy.ndarray[DTYPE_t, ndim=4] imgSlice
    cdef numpy.ndarray[DTYPE_t, ndim=4] kernel
    
    cdef img,fm,x,y
    #For each image in batch
    for img in xrange(batch_size):
        #For each feature map (output map)
        for fm in xrange(num_out_feat_maps):
            #For each x-dim in output
            '''
                Striding is taken care of by going through all input x dimensions with a stride, then putting them into the
                output file 
            '''
            for x in xrange(0,xdims,strides[0]):
                #For each y-dim in output
                for y in xrange(0,ydims,strides[1]):
                    #Get image slice from entire image, corresponds to kernel size, accross all channels.
                    if(pool == True):
                        imgSlice = image[img, fm, x:x+kxdim, y:y+kydim]
                        maxi, maxInd = max_and_argmax(imgSlice, keepdims_argmax=True)
                        #No need to div by strides
                        output[img, fm, x/strides[0], y/strides[1]] = maxi
                        #Add first corresponding max into G mat
                        G[img,fm,x+maxInd[0],y+maxInd[1]] = 1
                        
                    else:
                        imgSlice = image[img, :, x:x+kxdim, y:y+kydim]
                        #Get kernels accross all channels.
                        kernel = kernels[:, fm, :, :]
                        
                        '''
                            Do the dot product to get the position.
                            Divide by strides to get actual output position
                        '''
                        output[img, fm, x/strides[0], y/strides[1]] = numpy.dot(imgSlice.flatten(),kernel.flatten())
                
    return output,G




In [ ]:
#Baseline experiment
import numpy
import logging
import shelve
from mlp.dataset import MNISTDataProvider

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Standard Conv Linear for 10 epochs')
logger.info('Initialising data providers...')

train_dp = MNISTDataProvider(dset='train', batch_size=10, max_num_batches=100, randomize=True, conv_reshape = True )
valid_dp = MNISTDataProvider(dset='valid', batch_size=1000, max_num_batches=-10, randomize=False, conv_reshape = True)
test_dp = MNISTDataProvider(dset='eval', batch_size=1000, max_num_batches=-10, randomize=False, conv_reshape = True)

from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.conv import ConvLinear, ConvMaxPool2D
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed

logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])


#Open file to save to
shelve_r = shelve.open("naiveLinConV", writeback = True)


#some hyper-parameters
nhid = 800
learning_rate = 0.5
max_epochs = 30
cost = CECost()
    
stats = []

train_dp.reset()
valid_dp.reset()
test_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(ConvLinear(num_inp_feat_maps = 1, num_out_feat_maps = 2, irange=0.2, rng=rng))
model.add_layer(ConvMaxPool2D(num_feat_maps = 2, conv_shape = (24,24)))
model.add_layer(Sigmoid(idim=2*12*12, odim=2*12*12, irange=0.2, rng=rng))
model.add_layer(Softmax(idim=2*12*12, odim=10, rng=rng))

# define the optimiser, here stochasitc gradient descent
# with fixed learning rate and max_epochs
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')
tr_stats, valid_stats = optimiser.train(model, train_dp, valid_dp)

logger.info('Testing the model on test set:')
tst_cost, tst_accuracy = optimiser.validate(model, test_dp)
logger.info('MNIST test set accuracy is %.2f %%, cost (%s) is %.3f'%(tst_accuracy*100., cost.get_name(), tst_cost))

stats.append((tr_stats, valid_stats, (tst_cost, tst_accuracy)))

shelve_r['results'] = (tr_stats, valid_stats, (tst_cost, tst_accuracy))

shelve_r.close()

INFO:root:Standard Conv Linear for 10 epochs
INFO:root:Initialising data providers...
INFO:root:Training started...
INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 2.385. Accuracy is 10.60%


In [4]:
with open('scl.log', 'r') as log_file:
    print log_file.read()

root:INFO Standard Conv Linear for 10 epochs
root:INFO Initialising data providers...
root:INFO Standard Conv Linear for 10 epochs
root:INFO Initialising data providers...
root:INFO Training started...
mlp.optimisers:INFO Epoch 0: Training cost (ce) for initial model is 2.352. Accuracy is 10.60%
mlp.optimisers:INFO Epoch 0: Validation cost (ce) for initial model is 2.369. Accuracy is 9.91%
mlp.optimisers:INFO Epoch 1: Training cost (ce) is 2.818. Accuracy is 8.80%
mlp.optimisers:INFO Epoch 1: Validation cost (ce) is 2.310. Accuracy is 9.90%
mlp.optimisers:INFO Epoch 1: Took 296 seconds. Training speed 23 pps. Validation speed 40 pps.
mlp.optimisers:INFO Epoch 2: Training cost (ce) is 2.314. Accuracy is 10.30%
mlp.optimisers:INFO Epoch 2: Validation cost (ce) is 2.303. Accuracy is 12.94%
mlp.optimisers:INFO Epoch 2: Took 296 seconds. Training speed 23 pps. Validation speed 40 pps.
mlp.optimisers:INFO Epoch 3: Training cost (ce) is 2.183. Accuracy is 17.60%
mlp.optimisers:INFO Epoch 3: V